# Ngoc Ha

# Task 3 - QC mentorship screening

# QAOA-encoding-kernel SVM

In [1]:
import numpy as np
import torch
from torch.nn.functional import relu

from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import pennylane as qml
from pennylane.templates import AngleEmbedding, AmplitudeEmbedding, QAOAEmbedding
from pennylane.operation import Tensor

from time import time

import matplotlib.pyplot as plt

np.random.seed(2023)

In [4]:
X, y = load_iris(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [5]:
n_qubits = len(X[0])
n_qubits

4

### The parametrized kernel is computed by taking inner product after QAOA embedding
### Inner product is computed using the "adjoint method"
### QAOA embedding circuit uses 2 layers, 4 features, 4 wires

In [6]:
dev_kernel = qml.device('default.qubit', wires=4)

@qml.qnode(dev_kernel, interface = "autograd")
def kernel(x1, x2):
    projector = np.zeros((2**n_qubits, 2**n_qubits))
    projector[0, 0] = 1
    shape = qml.QAOAEmbedding.shape(n_layers=2, n_wires=4)
    weights = np.random.random(shape)
    QAOAEmbedding(features=x1, weights=weights, wires=range(4))
    qml.adjoint(QAOAEmbedding)(features=x2, weights=weights, wires=range(4))
    return qml.expval(qml.Hermitian(projector, wires=range(n_qubits)))

In [7]:
kernel(X_train[0], X_train[0]) # sanity check, should return 1

tensor(1., requires_grad=True)

In [8]:
def kernel_matrix(A, B):
    """Compute the matrix whose entries are the kernel
       evaluated on pairwise data from sets A and B."""
    return np.array([[kernel(a, b) for b in B] for a in A])

In [9]:
start = time()
svm = SVC(kernel=kernel_matrix).fit(X_train, y_train)
end = time()

Training time: -132.224591255188


In [11]:
print("Training time:", end-start)

Training time: 132.224591255188


In [12]:
predictions = svm.predict(X_test)
accuracy_score(predictions, y_test)

0.9473684210526315

# Angle-encoding-kernel SVM

In [13]:
# scaling the inputs is important since the embedding we use is periodic
scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)

# don't need to scale labels as sklearn will take care of them
y_scaled = y

X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled = train_test_split(X_scaled, y_scaled)

In [14]:
dev_kernel_2 = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev_kernel_2, interface="autograd")
def kernel_2(x1, x2):
    """Angle-encoding-based kernel"""
    projector = np.zeros((2**n_qubits, 2**n_qubits))
    projector[0, 0] = 1
    AngleEmbedding(x1, wires=range(n_qubits))
    qml.adjoint(AngleEmbedding)(x2, wires=range(n_qubits))
    return qml.expval(qml.Hermitian(projector, wires=range(n_qubits)))

In [15]:
kernel_2(X_train_scaled[0], X_train_scaled[0]) # sanity check, should return 1

tensor(1., requires_grad=True)

In [16]:
def kernel_matrix_2(A, B):
    """Compute the matrix whose entries are the kernel
       evaluated on pairwise data from sets A and B."""
    return np.array([[kernel_2(a, b) for b in B] for a in A])

In [17]:
start_2 = time()
svm_2 = SVC(kernel=kernel_matrix_2).fit(X_train_scaled, y_train_scaled)
end_2 = time()

In [18]:
print("Training time:", end_2-start_2)

Training time: 40.013169050216675


In [19]:
predictions_2 = svm_2.predict(X_test_scaled)
accuracy_score(predictions_2, y_test_scaled)

0.9736842105263158

#### SVM with an angle-encoding-based kernel outperforms SVM using a QAOA-embedding-based kernel (acc: 0.974 vs 0.947), while needing much less training time (40s vs 132s) and requiring the same number of qubits (4).

(This result is specific for QAOA embedding with 2 layers, 4 features, 4 wires and seed = 2023.)